In [1]:
from dotenv import load_dotenv
import requests, os
import pandas as pd
from requests.exceptions import RequestException, Timeout
load_dotenv()

api_key = os.getenv('riot_api_key')
player_url = os.getenv('url')
player_puuid = os.getenv('puuid_url')
match_url = os.getenv('match_url')
rank = os.getenv('tft_rank')

In [2]:
#define a função, passando a url e o timeout.
#Começa o try/except:
#faz a requisição, passando como parâmetro a url da função e o timeout.
#use o raise_for_status para tratar qualquer erro.
#retorna o json do respose.
#faz o except do timeout.
#printa que foi timeout com fstring e passa a url pra ajudar a identificar onde o código quebrou
#não retorne nada.
#outro except pro request exception(erro de requisição.)
#passa o print especificando que o erro foi em x lugar.

In [3]:
#request handler
def get_safe(url, timeout=20):
    try:
        response = requests.get(url, timeout=timeout)
        response.raise_for_status()
        return response.json()
    
    except Timeout:
        print(f'Conection Timeout: {url}')
        return None
    except RequestException as e:
        print(f'Bad request: {e}')
        return None

In [4]:
#raw data from api.

def fetch_matches():
    gameName = input('Type in your in-game name: ')
    tagLine = input('Type in your tagline: ')

    player_url_edited = player_url.format(gameName=gameName, tagLine=tagLine, api_key=api_key)

    response_puuid = get_safe(player_url_edited)
    
    if not response_puuid:
        return None

    puuid = response_puuid["puuid"]

    player_tft = player_puuid.format(puuid=puuid, api_key=api_key)

    response_tft = get_safe(player_tft)

    if not response_tft:
        return None

    match_url_edited = match_url.format(puuid=puuid, api_key=api_key)
    response_match = get_safe(match_url_edited)

    if not response_match:
        return None
    
    rank_edited = rank.format(puuid=puuid, api_key=api_key)

    response_rank = get_safe(rank_edited)

    if not response_rank:
        return None

    all_matches = []

    match_details = os.getenv('match_dets')

    for match_id in response_match:
        match_url_formatted = match_details.format(
            match_id=match_id,
            api_key=api_key
        )

        match_data = get_safe(match_url_formatted)

        if match_data:
            all_matches.append(match_data)


    return all_matches, puuid, gameName, response_rank # SE VOCE VAI RETORNAR QUATRO COISAS AQUI, TEM QUE CAPTURAR DENOVO NA CELL ABAIXO!!!!



In [5]:


matches, my_puuid, gameName, response_rank = fetch_matches() # TIPO ASSIM!!!!!


def get_details(matches, my_puuid):
    details = []

    for match in matches:
        for player in match['info']['participants']:
            if player['puuid'] == my_puuid:
                
                placements = player['placement']
                companion = player['companion']

                details.append({
                    'placement': player['placement'],
                    'companion': player['companion'],
                    })
                break

    return details

details = get_details(matches, my_puuid)
df = pd.DataFrame(details)
print(df)

    

   placement                                          companion
0          1  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
1          5  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
2          5  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
3          3  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
4          6  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
5          7  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
6          3  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
7          6  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
8          4  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
9          1  {'content_ID': 'f686392e-6431-4d32-bde0-8c7ee6...
